In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
import cv2
import random
import matplotlib.image as mpimg

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import keras
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.client import device_lib


from skimage.transform import resize
import imageio
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

class ImgMaskGenerator(object):
    def __init__(self,
                 dirpath='./',
                 img_w=None, img_h=None,
                 batch_size=3,
                 img_c = 3,
                 verbose=1):
        
        # configurations    
        self.HEIGHT     = img_h
        self.WIDTH      = img_w
        self.BATCH_SIZE = batch_size
        self.COLORS     = img_c
        self.DIRPATH    = dirpath
    
    def augaug(self, partSize = 100):
        ia.seed(1)
        images = []
        segmaps = []
        
        self.ids = self.ids_train + self.ids_valid
        for n, _id in enumerate(self.ids):
            print(n, _id)
            # Load images
            img = load_img(os.path.join(self.IMAGES_DIR, _id))
            images.append(img_to_array(img))
    

            # Load masks
            mask = img_to_array(load_img(os.path.join(self.MASKS_DIR, _id)))
            
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15)) # find coef
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            
            rhash = random.getrandbits(128)      
            mpimg.imsave("augdataset/images/{}.jpeg".format(rhash), x_img/255)
            mpimg.imsave("augdataset/masks/{}.jpeg".format(rhash), mask/255)
            
            segmaps.append(mask)
            
            if len(images) == partSize:
                augmenters_imgs = [
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-45, 45),
                        shear=(-16, 16),
                        order=[0, 1]
                        #cval=(0, 255),
                        #mode=ia.ALL
                    ),
                    iaa.Crop(px=(0, 10)), # crop images from each side by 0 to 16px (randomly chosen)
                    iaa.Fliplr(0.5), # horizontally flip 50% of the images
                    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
                ]      
                
                seq_imgs = iaa.Sequential(augmenters_imgs, random_order=False)        
                seq_imgs_deterministic = seq_imgs.to_deterministic()
                

                imgs_aug = seq_imgs_deterministic.augment_images(images)
                
                masks_aug = seq_imgs_deterministic.augment_images(segmaps)
                
                
                for ima, ma in zip(imgs_aug, masks_aug):
                    rhash = random.getrandbits(128)
                    
                    mpimg.imsave("augdataset/images/{}.jpeg".format(rhash), ima/255)
                    mpimg.imsave("augdataset/masks/{}.jpeg".format(rhash), ma/255)
                
                images  = []
                segmaps = []
        
    def prepare_data(self, verbose=1):
        # check paths
        self.IMAGES_DIR = os.path.join(self.DIRPATH, "images")
        self.MASKS_DIR = os.path.join(self.DIRPATH, "masks")
        if not os.path.exists(self.IMAGES_DIR):
            raise Exception("Path not exists {}!".format(self.IMAGES_DIR))
        if not os.path.exists(self.MASKS_DIR):
            raise Exception("Path not exists {}!".format(self.MASKS_DIR))
        
        # dataset
        ids = next(os.walk(self.IMAGES_DIR))[2] # list of names all images in the given path
        
        # Split on train and valid
        self.ids_train, self.ids_valid = train_test_split(ids, test_size=0.05, random_state=66)
        self.N_train = (len(self.ids_train) // self.BATCH_SIZE) + 1
        self.N_valid = (len(self.ids_valid) // self.BATCH_SIZE) + 1
        if verbose:
            print("Train: ", self.N_train, " * ", self.BATCH_SIZE)
            print("Valid: ", self.N_valid, " * ", self.BATCH_SIZE)
    
    def normalize(self, x_img):
        return resize(x_img, (self.HEIGHT, self.WIDTH, self.COLORS))
    
    def load_all_data(self, mode="valid"):
        assert mode in ["train", "valid"]
        buff = []
        if mode == "train":
            self.ids = self.ids_train
        else:
            self.ids = self.ids_valid
        
        self.ids = self.ids_valid
        X = np.zeros((len(self.ids), self.HEIGHT, self.WIDTH, self.COLORS), dtype=np.float32)
        y = np.zeros((len(self.ids), self.HEIGHT, self.WIDTH, 2), dtype=np.float32) 
        
        for n, _id in enumerate(self.ids):
            # Load images
            img = load_img(os.path.join(self.IMAGES_DIR, _id))
            x_img = img_to_array(img)
            x_img = self.normalize(x_img)

            # Load masks
            mask = img_to_array(load_img(os.path.join(self.MASKS_DIR, _id), grayscale=True))
            mask = resize(mask, (self.HEIGHT, self.WIDTH, 1), mode = 'constant', preserve_range = True)
            # Save images
            X[n] = x_img/255.0
            y[n] = mask/255.0
            
        return X, y
        
    def generator(self, partSize=20, mode="train"):
        assert mode in ["train", "valid"]
            
        buff = []
        if mode == "train":
            self.ids = self.ids_train
        else:
            self.ids = self.ids_valid
        
        parts = []
        buff = []
        X = np.zeros((self.BATCH_SIZE, self.HEIGHT, self.WIDTH, self.COLORS), dtype=np.float32)
        y = np.zeros((self.BATCH_SIZE, self.HEIGHT, self.WIDTH, 2), dtype=np.float32)    
        while True:
            for x in self.ids:
                buff.append(x)
                if len(buff)  == self.BATCH_SIZE:
                    for n, _id in enumerate(buff):
                        # Load images
                        img = load_img(os.path.join(self.IMAGES_DIR, _id))
                        
                        try:
                            x_img = img_to_array(img)
                        except:
                            print("ERROR: ",buff)
                            break
                            buff = []
                            
                        mask = img_to_array(load_img(os.path.join(self.MASKS_DIR, _id), grayscale=True))
                        
                        x_img = self.normalize(x_img)
            
                        # Load masks
                        mask = resize(mask, (self.HEIGHT, self.WIDTH, 1), mode = 'constant', preserve_range = True)
                        # Save images
                        X[n] = x_img/255.0
                        #print(keras.utils.to_categorical(mask/255.0, num_classes=2).shape)
                        #print((mask/255.0).shape)
                        #print((y[n]).shape)
                        y[n] = keras.utils.to_categorical(mask/255.0, num_classes=2)
                        #print(mask)
                        
                    parts.append((X, y))
                    if len(parts) > partSize:
                        for part in parts:
                            yield part
                        parts = []
                        
                    # to default 
                    X = np.zeros((self.BATCH_SIZE, self.HEIGHT, self.WIDTH, self.COLORS), dtype=np.float32)
                    y = np.zeros((self.BATCH_SIZE, self.HEIGHT, self.WIDTH, 2), dtype=np.float32)
                    buff = []
                    
class SDetector(ImgMaskGenerator):
    def __init__(self, config={}):
        # configurations    
        self.HEIGHT     = config.get("img_h", 32*48)
        self.WIDTH      = config.get("img_w", 32*48)
        self.BATCH_SIZE = config.get("batch_size", 5)
        self.COLORS     = config.get("img_c", 3)
        
        self.EPOCHS     = config.get("epochs", 50)
        self.COUNT_CATEGORIES = config.get("count_categories", 2)
        
    def conv_block(self, inputs, conv_type, kernel, kernel_size, strides, padding='same', relu=True):
        if(conv_type == 'ds'):
            x = keras.layers.SeparableConv2D(kernel, kernel_size, padding=padding, strides = strides)(inputs)
        else:
            x = keras.layers.Conv2D(kernel, kernel_size, padding=padding, strides = strides)(inputs)  

        x = keras.layers.BatchNormalization()(x)

        if (relu):
            x = keras.layers.Activation('relu')(x)

        return x
    
    def _res_bottleneck(self, inputs, filters, kernel, t, s, r=False):
        tchannel = keras.backend.int_shape(inputs)[-1] * t

        x = self.conv_block(inputs, 'conv', tchannel, (1, 1), strides=(1, 1))

        x = keras.layers.DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation('relu')(x)

        x = self.conv_block(x, 'conv', filters, (1, 1), strides=(1, 1), padding='same', relu=False)

        if r:
            x = keras.layers.add([x, inputs])
        return x
    
    def bottleneck_block(self, inputs, filters, kernel, t, strides, n):
        x = self._res_bottleneck(inputs, filters, kernel, t, strides)
        for i in range(1, n):
            x = self._res_bottleneck(x, filters, kernel, t, 1, True)

        return x
    
    def pyramid_pooling_block(self, input_tensor, bin_sizes, w, h):
        concat_list = [input_tensor]

        for bin_size in bin_sizes:
            pool_strides_size=(w//bin_size, h//bin_size)

            x = keras.layers.AveragePooling2D(pool_size=pool_strides_size, strides=pool_strides_size)(input_tensor)
            x = keras.layers.Conv2D(128, 3, strides=2, padding='same')(x)
            x = keras.layers.Lambda(lambda x: keras.backend.tf.image.resize_images(x, (w,h)))(x)

            concat_list.append(x)

        return keras.layers.concatenate(concat_list)

    def get_scnn(self, input_layer, h, w):
        """Function to define the SCNN Model"""
        assert h%32 == 0
        assert w%32 == 0
        
        lds_layer = self.conv_block(input_layer, 'conv', w//32, (3, 3), strides = (2, 2))
        lds_layer = self.conv_block(lds_layer, 'ds', w//32 + (w//32)//2, (3, 3), strides = (2, 2))
        lds_layer = self.conv_block(lds_layer, 'ds', 2*(w//32), (3, 3), strides = (2, 2))

        gfe_layer = self.bottleneck_block(lds_layer, h//32, (3, 3), t=6, strides=2, n=3)
        gfe_layer = self.bottleneck_block(gfe_layer, h//32 + (h//32)//2, (3, 3), t=6, strides=2, n=3)
        gfe_layer = self.bottleneck_block(gfe_layer, 2*(h//32), (3, 3), t=6, strides=1, n=3)
        gfe_layer = self.pyramid_pooling_block(gfe_layer, [2,4,6,8], h//32, w//32)

        ff_layer1 = self.conv_block(lds_layer, 'conv', 2*(h//32), (1,1), padding='same', strides= (1,1), relu=False)
        ff_layer2 = keras.layers.UpSampling2D((4, 4))(gfe_layer)
        ff_layer2 = keras.layers.SeparableConv2D(2*(h//32), (3, 3), padding='same', strides = (1, 1), activation=None, dilation_rate=(4, 4))(ff_layer2)

        ff_final = keras.layers.add([ff_layer1, ff_layer2])
        ff_final = keras.layers.BatchNormalization()(ff_final)
        ff_final = keras.layers.Activation('relu')(ff_final)

        classifier = keras.layers.SeparableConv2D(2*(h//32), (3, 3), padding='same', strides = (1, 1), name = 'DSConv1_classifier')(ff_final)
        classifier = keras.layers.BatchNormalization()(classifier)
        classifier = keras.layers.Activation('relu')(classifier)

        classifier = keras.layers.SeparableConv2D(2*(h//32), (3, 3), padding='same', strides = (1, 1), name = 'DSConv2_classifier')(classifier)
        classifier = keras.layers.BatchNormalization()(classifier)
        classifier = keras.layers.Activation('relu')(classifier)

        classifier = self.conv_block(classifier, 'conv', self.COUNT_CATEGORIES, (1, 1), strides=(1, 1), padding='same', relu=True)

        classifier = keras.layers.Dropout(0.3)(classifier)

        classifier = keras.layers.UpSampling2D((8, 8))(classifier)
        classifier = keras.layers.Activation('softmax')(classifier)

        self.model = keras.Model(inputs = input_layer , outputs = classifier, name = 'Fast_SCNN')
   
        return self.model
    
    def predict(self, imgs, shapes):
        X = np.zeros((len(imgs), self.HEIGHT, self.WIDTH, self.COLORS), dtype=np.float32)
        for i, img in enumerate(imgs):
            X[i]      = self.normalize(img)
            
            #print(X[i])
            plt.imshow(X[i])
            plt.show()
            
            shapes[i] = np.array((img.shape[0], img.shape[1], 3))
     
        preds = self.model.predict(X)
        
        preds_t = preds[:, :, :, 1:2].reshape((preds.shape[0],preds.shape[1], preds.shape[2]))
        
        return (preds_t > 0.2).astype(np.uint8)
        
    def detect(self, imgs):
        shapes = np.zeros((len(imgs), 3))
        preds_t = self.predict(imgs, shapes)
        res = []
        
        for j, (pred, shape) in enumerate(zip(preds_t, shapes)):
            img = imgs[j]
            preds_t_n = []
            
            pred = cv2.cvtColor(pred, cv2.COLOR_GRAY2RGB)*255

            # clear mask
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15)) # find coef
            pred = cv2.morphologyEx(pred, cv2.MORPH_OPEN, kernel)
            pred = cv2.morphologyEx(pred, cv2.MORPH_CLOSE, kernel)

            pred = resize(pred, shape).astype(np.float32)

            pred = cv2.cvtColor(pred*255, cv2.COLOR_RGB2GRAY)

            ret, thresh = cv2.threshold(pred.astype(np.uint8), 127, 255, 0)
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            for i, c in enumerate(contours):
#                 if (cv2.contourArea(c)/(img.shape[0]*img.shape[1])) < 0.0035:
#                     print("BAD AREA")
#                     # detect center
#                     M = cv2.moments(c)
#                     cX = int(M["m10"] / M["m00"])
#                     cY = int(M["m01"] / M["m00"])
                    
#                     refineSize = 256
#                     x1 = cX - refineSize if cX - refineSize > 0 else 0
#                     y1 = cY - refineSize if cY - refineSize > 0 else 0
                    
#                     x2 = cX + refineSize if cX + refineSize < img.shape[1] else img.shape[1]
#                     y2 = cY + refineSize if cY + refineSize < img.shape[0] else img.shape[0]
    
#                     small_img = img[y1:y2, x1:x2]
                    
#                     small_shapes = np.zeros((len(imgs), 3))
#                     preds_t = self.predict([small_img], small_shapes)
#                     pred = cv2.cvtColor(preds_t[0], cv2.COLOR_GRAY2RGB)*255
#                     pred = cv2.morphologyEx(pred, cv2.MORPH_OPEN, kernel)
                    
#                     pred = cv2.morphologyEx(pred, cv2.MORPH_CLOSE, kernel)

#                     pred = resize(pred, small_shapes[0]).astype(np.float32)

#                     pred = cv2.cvtColor(pred*255, cv2.COLOR_RGB2GRAY)

#                     ret, thresh = cv2.threshold(pred.astype(np.uint8), 127, 255, 0)
#                     contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                    
#                     #cv2.drawContours(small_img, contours, -1, (0, 255, 0), 2)
#                     #plt.imshow(small_img)
#                     #plt.show()
#                     if len(contours):
#                         c = contours[0]
#                         c = np.array([[[ic[0][0] + x1, ic[0][1] + y1]] for ic in c])
                           
                new_img = np.zeros(img.shape, np.uint8)
                cv2.fillConvexPoly(new_img, np.array(cv2.convexHull(c), 'int32'), (255, 255, 255))
                preds_t_n.append(new_img)
            res.append(preds_t_n)

        return res
    
    def create_model(self):
        # Input Layer
        input_layer = keras.layers.Input(shape=((self.HEIGHT, self.WIDTH, self.COLORS)), name = 'input_layer')
        self.model = self.get_scnn(input_layer, self.HEIGHT, self.WIDTH)
        
        optimizer = keras.optimizers.SGD(momentum=0.9, lr=0.045)
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        return self.model
    
    def prepare(self, dirpath, verbose=1):
        self.DIRPATH = dirpath
        self.prepare_data(verbose=verbose)
        
        self.train_gen = self.generator(mode="train")
        self.valid_gen = self.generator(mode="valid")
        
    def fit(self, tmp_path='./tmp_scnn.h5', epochs=None, verbose=1):
        self.callbacks = [
            EarlyStopping(patience=20, verbose=verbose),
            ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=verbose),
            ModelCheckpoint(tmp_path, verbose=verbose, save_best_only=True, save_weights_only=True)
        ]
        return self.model.fit_generator(    self.train_gen, 
                      steps_per_epoch     = self.N_train, 
                      epochs              = epochs or self.EPOCHS, 
                      verbose             = verbose, 
                      callbacks           = self.callbacks, 
                      validation_data     = self.valid_gen, 
                      validation_steps    = self.N_valid)
    
    def load_last(self, tmp_path='./tmp_scnn.h5'):
        self.model.load_weights(tmp_path)
        
    def save(self, path):
        self.model.save(path)
        return path

    def load(self, path):
        self.model = load_model(path)
        return self.model

Using TensorFlow backend.


# Model Architecture

![Fast-SCNN Architecture](https://github.com/DeepVoltaire/Fast-SCNN/raw/master/figures/fast-scnn.png)

In [2]:
detector = SDetector({"epochs": 100})
model = detector.create_model()

W0606 10:11:49.062182 140136009078400 deprecation_wrapper.py:119] From /usr/local/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0606 10:11:49.070183 140136009078400 deprecation_wrapper.py:119] From /usr/local/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0606 10:11:49.070744 140136009078400 deprecation_wrapper.py:119] From /usr/local/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0606 10:11:49.083797 140136009078400 deprecation_wrapper.py:119] From /usr/local/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0606 10:11:49.084109 14

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 1536, 1536, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 768, 768, 48) 1344        input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 768, 768, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 768, 768, 48) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_

In [4]:
detector.prepare("./augdataset")

Train:  996  *  5
Valid:  53  *  5


In [ ]:
detector.fit()

Epoch 1/100


/usr/local/lib64/python3.7/site-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


996/996 [==============================] - 1455s 1s/step - loss: 0.2320 - acc: 0.9941 - val_loss: 0.0257 - val_acc: 0.9956

Epoch 00001: val_loss improved from inf to 0.02570, saving model to ./tmp_scnn.h5
Epoch 2/100
996/996 [==============================] - 1423s 1s/step - loss: 0.2132 - acc: 0.9956 - val_loss: 0.0136 - val_acc: 0.9956

Epoch 00002: val_loss improved from 0.02570 to 0.01359, saving model to ./tmp_scnn.h5
Epoch 3/100
996/996 [==============================] - 1420s 1s/step - loss: 0.2117 - acc: 0.9956 - val_loss: 0.0116 - val_acc: 0.9956

Epoch 00003: val_loss improved from 0.01359 to 0.01159, saving model to ./tmp_scnn.h5
Epoch 4/100
996/996 [==============================] - 1424s 1s/step - loss: 0.2113 - acc: 0.9956 - val_loss: 0.0116 - val_acc: 0.9956

Epoch 00004: val_loss did not improve from 0.01159
Epoch 5/100
996/996 [==============================] - 1423s 1s/step - loss: 0.2112 - acc: 0.9956 - val_loss: 0.0113 - val_acc: 0.9956

Epoch 00005: val_loss impro

In [5]:
detector.load_last(tmp_path='./tmp_scnn.h5')

In [ ]:
X, y = detector.load_all_data(mode="valid")

/usr/local/lib64/python3.7/site-packages/keras_preprocessing/image/utils.py:98: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
# crop
X, y = X[:20], y[:20]

In [ ]:
detector.model.evaluate(X, y, verbose=1)

In [ ]:
preds = detector.model.predict(X, verbose=1)

In [ ]:
preds_t = (preds > 0.5).astype(np.uint8)

In [ ]:
preds = preds[:, :, :, 1:2]

In [ ]:
# get second class
preds_t = preds_t[:, :, :, 1:2] 
y = y[:, :, :, 1:2] 

In [ ]:
def plot_sample(X, y, preds, binary_preds, ix=None):
    """Function to plot the results"""
    if ix is None:
        ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ...])
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Orig')

    ax[1].imshow(y[ix].squeeze())
    ax[1].set_title('Np')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('NP Predicted')
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[3].set_title('NP Predicted binary');

In [ ]:
%%time
# Check if training data looks all right
plot_sample(X, y, preds, preds_t, ix=14)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
%%time
plot_sample(X, y, preds, preds_t)

In [ ]:
2048//32 + (2048//32)